### **Proje Yöntemi**

Bu projede, DNA dizilimleri gibi sıralı (sekans) yapıya sahip verilerin modellenmesinde etkin sonuçlar üretmesi nedeniyle Uzun Kısa Süreli Bellek (Long Short-Term Memory, LSTM) tabanlı bir derin öğrenme mimarisi kullanılmıştır. LSTM modeli, dizisel bağımlılıkları öğrenme yeteneği sayesinde biyolojik dizilerin analizinde tercih edilmiştir.

Kullanılan veri seti sınıf dağılımı açısından dengeli olmasına rağmen, modelin genelleme kabiliyetini artırmak amacıyla veri çoğaltma (data augmentation) yöntemi uygulanmıştır. Ham veri setinde etiket bilgisi eksik olan örnekler tespit edilerek analiz dışı bırakılmıştır. Ardından, her bir DNA dizisi için etiketler tanımlanmış ve nükleotitler (A, T, C, G) sayısal temsillere dönüştürülerek modele uygun hale getirilmiştir.

Değişken uzunluktaki dizilerin modele tutarlı bir biçimde aktarılabilmesi için tüm dizilere sabit uzunluk kazandırmak amacıyla padding işlemi uygulanmıştır. Eğitim sürecinde, DNA dizilerinin daha anlamlı ve düşük boyutlu vektör temsillerine dönüştürülmesi amacıyla Embedding katmanı kullanılmış; bu yaklaşım aynı zamanda modelin öğrenme sürecini hızlandırmıştır.

Hesaplama maliyetini ve bellek kullanımını azaltmak amacıyla, veri seti modele parça parça aktarılan *chunking* yöntemiyle işlenmiştir. Önceden uygulanan padding işlemi nedeniyle oluşan boş veri parçalarının (empty chunks) modele dahil edilmemesi için ek bir kontrol mekanizması geliştirilmiştir.

Modelin performansını değerlendirmek için kayıp fonksiyonu olarak *binary cross-entropy*, optimizasyon algoritması olarak ise RMSprop tercih edilmiştir. Kod yapısı, her bir işlem adımının ayrı fonksiyonlar halinde tanımlanmasıyla oluşturulmuş; bu sayede kodun okunabilirliği artırılmış ve ilerleyen aşamalarda yapılacak düzenlemeler kolaylaştırılmıştır.

Model geliştirme sürecinde, başlangıçta temel bir mimari oluşturulmuş ve elde edilen sonuçlara bağlı olarak epoch sayısı, öğrenme oranı (*learning rate*), batch size ve maksimum dizi uzunluğu (*max sequence length*) gibi hiperparametreler üzerinde çeşitli denemeler yapılmıştır. Elde edilen sonuçların beklenen seviyeye ulaşamamasının temel nedeninin veri setinin büyüklüğü olduğu düşünülmektedir. Her ne kadar veri seti dengeli bir yapıya sahip olsa da, yalnızca 80 protein dizilimi içermektedir. Daha büyük ölçekli, yaklaşık bin protein dizilimini kapsayan bir veri seti ile modelin performansının anlamlı ölçüde artacağı öngörülmektedir.

---

### **Veri Seti Oluşturma Süreci (Veri Seti Hikâyesi)**

Bu çalışmada kullanılan veri seti, 40 adet fonksiyonel ve 40 adet fonksiyonel olmayan olmak üzere toplam 80 DNA diziliminden oluşmaktadır. Her bir dizilim yaklaşık 30.000 ila 50.000 nükleotit uzunluğuna sahiptir. Veriler, farklı biyoinformatik kaynaklardan ve toplam 80 ayrı siteden elde edilerek derlenmiştir. Bu süreç sonucunda, sınıflar arasında dengeli bir dağılıma sahip özgün bir veri seti oluşturulmuştur.

---

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.metrics import classification_report

# Veri yükleme
def load_data(file_path):
    # Belirtilen dosya yolundan veriyi yükler ve temel bilgiler verir.
    data = pd.read_csv(file_path)
    print("Veri başarıyla yüklendi!")
    print(f"Satır Sayısı: {data.shape[0]}, Sütun Sayısı: {data.shape[1]}")
    return data

# Veri keşfi
def explore_data(data):
    # Veri seti hakkında bilgi ve istatistikleri ekrana yazdırır.
    print("\nVeri Türleri:")
    print(data.info())
    print("\nEtiket Dağılımı:")
    print(data['Label'].value_counts())
    print("\nEksik Veri Sayısı:")
    print(data.isnull().sum())

# Veri artırma (class balancing)
def balance_data(data):
    # Veri setindeki sınıfların dengesini sağlar.
    class_0 = data[data['Label'] == 0]
    class_1 = data[data['Label'] == 1]

    # Az olan sınıfı artırarak dengeler.
    class_1_upsampled = resample(class_1,
                                 replace=True,
                                 n_samples=len(class_0),
                                 random_state=42)

    # Dengeli veri setini birleştirir.
    balanced_data = pd.concat([class_0, class_1_upsampled])
    print("\nVeri dengelendi. Yeni etiket dağılımı:")
    print(balanced_data['Label'].value_counts())
    return balanced_data.reset_index(drop=True)

# Veri ön işleme
def preprocess_data(data, max_sequence_length=1800):
    # Eksik etiketleri temizler ve DNA dizilerini uygun formatta işler.

    # Etiketlerde eksik olan satırları çıkarır.
    data = data.dropna(subset=['Label']).reset_index(drop=True)

    # Etiketleri sayısal değerlere kodlar.
    label_encoder = LabelEncoder()
    data.loc[:, 'Label'] = label_encoder.fit_transform(data['Label'])  # 0 ve 1 kodlaması

    # DNA dizilerini sayısal formatta kodlar.
    nucleotide_map = {'A': 1, 'T': 2, 'G': 3, 'C': 4}  # 0 padding için ayrıldı
    sequences_encoded = data['Sequence'].apply(
        lambda seq: [nucleotide_map[char] for char in seq]
    )

    # Dizileri aynı uzunluğa getirir (padding).
    if not max_sequence_length:
        max_sequence_length = max(sequences_encoded.apply(len))
    sequences_padded = np.array([
        seq + [0] * (max_sequence_length - len(seq)) if len(seq) < max_sequence_length else seq[:max_sequence_length]
        for seq in sequences_encoded
    ])

    return sequences_padded, data['Label'], max_sequence_length

# LSTM modeli oluşturma
def create_lstm_model(max_sequence_length):
    # LSTM tabanlı model oluşturur.
    model = Sequential([
        Embedding(input_dim=5, output_dim=64, input_length=max_sequence_length),
        LSTM(128, return_sequences=True),
        Dropout(0.5),
        LSTM(64),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # Modeli RMSprop optimizer ile derler.
    model.compile(optimizer=RMSprop(learning_rate=0.00001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Chunking ile veri işleme ve eğitim
def train_model_with_chunking(model, data, chunk_size, max_sequence_length, epochs, batch_size):
    # Veriyi parça parça işler ve modeli eğitir.Belleği yormamak için.
    num_chunks = len(data) // chunk_size + int(len(data) % chunk_size != 0)
    print(f"Toplam {num_chunks} chunk işlenecek.")

    #modeli chunkin ile eğitilmesi için döngü oluşturur
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        for i in range(0, len(data), chunk_size):
            chunk = data.iloc[i:i + chunk_size]
            X_chunk, y_chunk, _ = preprocess_data(chunk, max_sequence_length)
            y_chunk = np.array(y_chunk)

            # Chunk'ın dolu olup olmadığını kontrol eder
            if X_chunk.size == 0 or y_chunk.size == 0:
                print("Boş chunk atlandı.")
                continue

            # LSTM modeli için X_chunk'ın şekli
            X_chunk = np.array(X_chunk).reshape(-1, max_sequence_length)

            print(f"Chunk boyutu: {X_chunk.shape}, y_chunk boyutu: {y_chunk.shape}")

            # Modeli eğitir
            model.fit(X_chunk, y_chunk, batch_size=batch_size, epochs=1, verbose=1)
    return model

# Ana iş akışı
def main():
    file_path = 'data.txt'

    # 1. Veri Yükleme
    data = load_data(file_path)

    # 2. Veri Keşfi
    explore_data(data)

    # 3. Veri Dengesi
    data = balance_data(data)

    # 4. Veri Ön İşleme
    X, y, max_sequence_length = preprocess_data(data)

    # 5. Eğitim ve test setini bölme
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

    # 6. Model Oluşturma
    model = create_lstm_model(max_sequence_length)

    # 7. Chunking ile Model Eğitimi
    print("\nModel eğitiliyor...")
    trained_model = train_model_with_chunking(
        model, train_data, chunk_size=900, max_sequence_length=max_sequence_length, epochs=25, batch_size=11
    )

    # 8. Model Değerlendirme
    #test_data modelin anlayabileceği formata dönüştürülür.
    print("\nModel değerlendiriliyor...")
    X_test, y_test, _ = preprocess_data(test_data, max_sequence_length)
    y_test = np.array(y_test)

    #Olasılık değeri 0.5'ten büyükse, model bu DNA dizisinin "fonksiyonel" olduğunu tahmin eder.
    #Aksi halde "fonksiyonel değil" (0) tahmini yapılır. astype tahmin edilen değerleri tam sayıya (0 veya 1) dönüştürür.
    predictions = (trained_model.predict(X_test) > 0.5).astype("int32")

    # test veri seti için gerçek etiketlerle modelin tahminlerini karşılaştırır
    print(classification_report(y_test, predictions))
    loss, accuracy = trained_model.evaluate(X_test, y_test, verbose=1)
    print(f"\nTest Doğruluğu: {accuracy:.2f}")

# Çalıştırma
if __name__ == "__main__":
    main()

Veri başarıyla yüklendi!
Satır Sayısı: 45633, Sütun Sayısı: 2

Veri Türleri:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45633 entries, 0 to 45632
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sequence  45633 non-null  object 
 1   Label     80 non-null     float64
dtypes: float64(1), object(1)
memory usage: 713.1+ KB
None

Etiket Dağılımı:
Label
0.0    40
1.0    40
Name: count, dtype: int64

Eksik Veri Sayısı:
Sequence        0
Label       45553
dtype: int64

Veri dengelendi. Yeni etiket dağılımı:
Label
0.0    40
1.0    40
Name: count, dtype: int64

Model eğitiliyor...
Toplam 1 chunk işlenecek.

Epoch 1/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.6593 - loss: 0.6912

Epoch 2/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.4344 - loss: 0.6954

Epoch 3/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.5443 - loss: 0.6918

Epoch 4/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.4706 - loss: 0.6933

Epoch 5/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.4697 - loss: 0.6960

Epoch 6/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.5634 - loss: 0.6883

Epoch 7/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 28s 5s/step - accuracy: 0.4541 - loss: 0.6945

Epoch 8/25
Chunk boyutu: (64, 1800), y_chunk boyutu: (64,)
6/6 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.5814 - loss: 0.6918

Epoch 9/25


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 992ms/step - accuracy: 0.3125 - loss: 0.6973

Test Doğruluğu: 0.31


In [ ]:
'''VERİ SETİ BU SİTELERDEN OLUŞTURULMUŞTUR
https://www.ncbi.nlm.nih.gov/nuccore/NG_033787.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_031916.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_012177.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_015845.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_028088.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_046865.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_029095.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_009876.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_023279.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_008116.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_009215.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_012805.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JALYAP010000018.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAOCZR010000006.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAOCZR010000013.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JALYCG010000020.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAPKJG010000051.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JARWTQ010000032.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAOCZR010000023.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAOCZR010000034.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAOCZR010000039.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JABUOZ010000054.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/PP520311.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/PP521662.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_041796.3?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000050.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000058.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000019.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000027.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000042.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000060.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000063.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000069.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAVKLT010000075.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JARXWJ010000024.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_031823.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_VUMI01000042.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_VUMI01000048.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_VUMI01000057.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_VUMI01000064.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JACBYI010000053.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAXCVI010000005.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NG_031992.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAMWGN010000012.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/CAYFAD010000064.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAMWGN010000016.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NW_019212895.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/D64044.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/AF503504.2?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/OQ626729.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/KZ804422.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/AJYU02000107.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NMSE01000019.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MLCZ01000020.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/QXFV01000144.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/CAXKWB010105999.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAMWGO010000013.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JANXRJ010000030.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/NZ_JAXCVJ010000010.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/LTZY01000037.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/KV793378.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MCXF01000102.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MCYC01000036.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MCZD01000178.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MCZL01000104.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MDAW01000153.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MDBF01000109.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/POWH01000010.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/PDNQ01002977.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/PJQY01003117.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/PIFU01000042.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/AUSU01000465.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/AYVE01000029.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/CBTQ010000016.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/CH478980.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/JYRW01000044.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/JYUI01000046.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/MIIY01000044.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/LXGZ01000144.1?report=fasta
https://www.ncbi.nlm.nih.gov/nuccore/LWMZ01000098.1?report=fasta'''